In [488]:
from enum import Enum
from typing import *
from abc import abstractmethod, ABC

from mimetypes import init
from time import perf_counter, sleep

from functools import total_ordering
from inspect import trace
from logging import raiseExceptions
from operator import truediv

In [489]:

""" LEVEL 01 : STATE """


class State:
    """ ÉTAT APPLICATIF COURANT -> DYNAMIQUE """

    def __init__(self, parameters: 'State.Parameters' = 'State.Parameters()'):
        if isinstance(parameters, State.Parameters):
            self.__params: State.Parameters = parameters
        else:
            raise TypeError('Les paramètres passés doivent être de type State.Parameters.')
        self.__transitions: List[Transition] = list()

    """ ACCESSEURS """

    """ State est valide selon 2 critères
    * existe-il au moins 1 transition ?
    * est-ce que chacune des transitions listées est valide ?
    """

    @property
    def is_valid(self) -> bool:
        # critère 001 : ANNULÉ PAR LE PROF
        #if len(self.__transitions) < 1 and not self.is_terminal:
        #    raise Exception("Il faut au moins une transition à un état qui n'est pas terminal.")
        # critère 002
        for t in self.__transitions:
            if not t.is_valid:
                raise Exception("Une des transitions est invalide. Peut-être qu'il manque la condition.")
        return True

    ''' query '''

    @property
    def is_terminal(self) -> bool:
        return self.__params.terminal

    """ Requête en boucle
    * retourne la première Transition effective
    """

    @property
    def is_transiting(self) -> 'Transition':
        for t in self.__transitions:
            if t.is_transiting:
                return t
        return None

    @property
    def params(self) -> 'Parameters':
        return self.__params

    @property
    def transitions(self) -> List['Transition']:
        return self.__transitions

    ''' MÉTHODES '''

    def add_transition(self, t: 'Transition'):
        self.__transitions.append(t)

    """ Exécution opérations connexes et propres à la mise en marche de State ( initialisation de l'état ) 
    * précède/ symbolise une entrée/ transition vers un State spécifié
    """

    def _exec_entering_action(self):
        self._do_entering_action()

        if self.__params.do_in_state_action_when_entering:
            self._exec_in_state_action()

    """ Exécution opérations propres au State actuel """

    def _exec_in_state_action(self):
        self._do_in_state_action()

    """ Exécution opérations connexes et propres à la mise en arrêt de State ( termination de l'état ) 
    * précède une sortie/ transition vers un autre State
    """

    def _exec_exiting_action(self):
        if self.__params.do_in_state_action_when_exiting:
            self._exec_in_state_action()

        self._do_exiting_action()

    def _do_entering_action(self):
        pass

    def _do_in_state_action(self):
        pass

    def _do_exiting_action(self):
        pass

    """ CLASSES INTERNES """

    class Parameters:
        def __init__(self):
            self.terminal: bool = False
            self.do_in_state_action_when_entering: bool = False
            self.do_in_state_action_when_exiting: bool = False


""" LEVEL 01 : TRANSITION """


class Transition(ABC):
    def __init__(self, next_state: 'State' = None) -> None:
        if isinstance(next_state, State) or next_state is None:
            self.__next_state = next_state
        else:
            raise TypeError("Le prochain état doit être de type State.")

    @property
    def is_valid(self) -> bool:
        return self.__next_state is not None

    @property
    def next_state(self) -> 'State':
        return self.__next_state

    @next_state.setter
    def next_state(self, value: 'State') -> None:
        if isinstance(value, State) or value is None:
            self.__next_state = value
        else:
            raise TypeError("Le prochain état doit être de type State.")

    @abstractmethod
    def is_transiting(self):
        pass

    def _exec_transiting_action(self) -> None:
        self._do_transiting_action()

    def _do_transiting_action(self) -> None:
        pass


""" LEVEL 01 : FINITE STATE MACHINE """

class FiniteStateMachine:
    class OperationState(Enum):
        UNITIALIZED = 0
        IDLE = 1
        RUNNING = 2
        TERMINAL_REACHED = 3

    # infrastucture statique des etat-transitions
    # comme feu de circulation (vert->jaune->rouge  ==> layout)
    class Layout:
        def __init__(self, states: List[State], initial_state: State) -> None:
            self.__states: List[State] = states
            if isinstance(initial_state, State):
                self.__initial_state: State = initial_state
            else:
                raise TypeError('Le paramètre doit être typé "State".')
            self.is_valid

        @property
        def is_valid(self) -> bool:
            if self.__initial_state is None:
                raise Exception('Il faut passer en paramètre un état initial de type State.')
            if self.__initial_state not in self.__states:
                raise Exception("Il faut que l'état initial soit dans la liste des états.")
            for state in self.__states:
                if not state.is_valid:
                    raise Exception("Une des transitions ne possède pas de prochain état.")
            return True

        @property
        def initial_state(self) -> State:
            return self.__initial_state

        @initial_state.setter
        def initial_state(self, value: State) -> None:
            if isinstance(value, State):
                self.__initial_state = value
            else:
                raise TypeError("Le paramètre doit être typé 'float'.")

        def add_state(self, state: State) -> None:
            if isinstance(state, State):
                self.__states.append(state)
            else:
                raise TypeError("L'état doit être de type State.")

        def add_states(self, statelist: List[State]) -> None:
            for state in statelist:
                if isinstance(state, State):
                    self.__states.append(state)
                else:
                    raise TypeError("Les états doivent être de type State")

    # A revoir
    def __init__(self, layout: Layout, uninitialized: bool = True) -> None:
        if isinstance(layout, FiniteStateMachine.Layout):
            self.__layout: FiniteStateMachine.Layout = layout
        else:
            raise TypeError('Layout not initialised')

        self.__current_applicative_state: State = self.__layout.initial_state
        self.__current_operational_state: FiniteStateMachine.OperationState = \
            FiniteStateMachine.OperationState.UNITIALIZED if uninitialized else FiniteStateMachine.OperationState.IDLE

    # l'action en cours de l'engin de resolution
    @property
    def current_operational_state(self) -> OperationState:
        return self.__current_operational_state

    # l'etat courant de la partie applicative
    @property
    def current_applicative_state(self) -> State:
        return self.__current_applicative_state

    def _transit_by(self, transition: Transition):
        self.__current_applicative_state._exec_exiting_action()
        transition._exec_transiting_action()
        self.__current_applicative_state = transition.next_state
        self.__current_applicative_state._exec_entering_action()

    def transit_to(self, state: State):
        self.__current_applicative_state = state
        self.current_applicative_state._exec_entering_action()
        
    def track(self) -> bool:
        # logique de fait un pas de calcul et validation de transition et appel de fonctions
        if self.__current_operational_state != FiniteStateMachine.OperationState.TERMINAL_REACHED:
            # Si le state est terminal, on indique qu'on entre en état terminal après avoir exécuté son in_state_action()
            if self.__current_applicative_state.params.terminal:
                self.__current_operational_state = FiniteStateMachine.OperationState.TERMINAL_REACHED
            transition = self.__current_applicative_state.is_transiting
            if transition is not None:
                self._transit_by(transition)
            else:
                self.__current_applicative_state._exec_in_state_action()
            return True
        else:
            return False

    def reset(self) -> OperationState:
        self.__current_operational_state = FiniteStateMachine.OperationState.IDLE
        return self.__current_operational_state

    def start(self, reset: bool = True, time_budget: float = None) -> None:
        timer_start = perf_counter()
        self.__current_operational_state = FiniteStateMachine.OperationState.RUNNING
        self.__current_applicative_state._exec_entering_action()
        while reset is True:
            # comteur de temps en seconde
            timer_total = perf_counter() - timer_start
            # print(self.__current_operational_state)
            if time_budget is not None:
                if not self.track() or timer_total > time_budget:
                    return
            else:
                if not self.track():
                    return

    def stop(self) -> OperationState:
        if self.__current_operational_state == FiniteStateMachine.OperationState.RUNNING:
            self.__current_operational_state = FiniteStateMachine.OperationState.IDLE
        return self.__current_operational_state

In [490]:

""" LEVEL 02 : STATE """


class ActionState(State):
    """ PERMET LA GÉNÉRATION DE State """

    def __init__(self, parameters: 'State.Parameters' = State.Parameters()):
        if not isinstance(parameters, State.Parameters):
            raise TypeError('Les paramètres passés doivent être de type State.Parameters.')
        super().__init__(parameters)
        ''' créer State par manipulation d'un Callable typé Action lorsque: entering, in_state, exiting, transiting '''
        self.__entering_actions: List[ActionState.Action] = list()
        self.__in_state_actions: List[ActionState.Action] = list()
        self.__exiting_actions: List[ActionState.Action] = list()

    Action = Callable[[], None]

    """ MÉTHODES """

    # @override
    def _do_entering_action(self):
        for action in self.__entering_actions:
            action()

    # @override
    def _do_in_state_action(self):
        for action in self.__in_state_actions:
            action()

    # @override
    def _do_exiting_action(self):
        for action in self.__exiting_actions:
            action()

    def add_entering_action(self, action: 'ActionState.Action'):
        if self.__is_callable_action(action):
            self.__entering_actions.append(action)

    def add_in_state_action(self, action: 'ActionState.Action'):
        if self.__is_callable_action(action):
            self.__in_state_actions.append(action)

    def add_exiting_action(self, action: 'ActionState.Action'):
        if self.__is_callable_action(action):
            self.__exiting_actions.append(action)

    @staticmethod
    def __is_callable_action(action: 'ActionState.Action') -> bool:
        if callable(action):
            return True
        else:
            raise RuntimeError("L'objet 'action' transmis en paramètre n'est pas un Callable.")


class MonitoredState(ActionState):
    """ UTILITAIRE
    * informations supplémentaires :
    - temps d'activité
    - compteurs

    * permet automatisation conditionnelle :
    - transition après un certain temps écoulé
    - transition après un certain nombre d'occurrences
    - transition selon une condition
    """

    def __init__(self, parameters: 'State.Parameters' = State.Parameters()):
        if not isinstance(parameters, State.Parameters):
            raise TypeError('Les paramètres passés doivent être de type State.Parameters.')
        super().__init__(parameters)
        self.__counter_last_entry: float = 0.
        self.__counter_last_exit: float = 0.
        self.__entry_count: int = 0
        self.custom_value: any

    """ ACCESSEURS """

    @property
    def last_entry_time(self) -> float:
        return self.__counter_last_entry

    @property
    def last_exit_time(self) -> float:
        return self.__counter_last_exit

    @property
    def entry_count(self) -> int:
        return self.__entry_count


    """ MUTATEURS """

    @last_entry_time.setter
    def last_entry_time(self, new_entry_time: float):
        if type(new_entry_time) is float:
            self.__counter_last_entry: float = new_entry_time
        else:
            raise TypeError("Le paramètre doit être typé 'float'.")

    @last_exit_time.setter
    def last_exit_time(self, new_exit_time: float):
        if type(new_exit_time) is float:
            self.__counter_last_exit: float = new_exit_time
        else:
            raise TypeError("Le paramètre doit être typé 'float'.")

    @entry_count.setter
    def entry_count(self, new_entry_count: int):
        if type(new_entry_count) is int:
            self.__entry_count: int = new_entry_count
        else:
            raise TypeError("Le paramètre doit être typé 'int'.")

    """ MÉTHODES """

    def reset_last_times(self):
        self.last_entry_time: float = 0.
        self.last_exit_time: float = 0.

    def reset_entry_count(self):
        self.entry_count: int = 0

    def _exec_entering_action(self):
        self.entry_count += 1
        self.last_entry_time = perf_counter()
        super()._exec_entering_action()

    def _exec_exiting_action(self):
        super()._exec_exiting_action()
        self.last_exit_time = perf_counter()
        


#########################


""" LEVEL 02 : TRANSITION """


class ConditionalTransition(Transition):
    def __init__(self, condition: 'Condition' = None) -> None:
        super().__init__()
        if isinstance(condition, Condition) or condition is None:
            self.__condition = condition
        else:
            raise TypeError("Le type de la condition doit être Condition.")

    @property
    def is_valid(self) -> bool:
        return self.__condition is not None

    @property
    def condition(self) -> 'Condition':
        return self.__condition

    @condition.setter
    def condition(self, value: 'Condition') -> None:
        if isinstance(value, Condition) or value is None:
            self.__condition = value
        else:
            raise TypeError("La condition doit être de type Condition.")

    @property
    def is_transiting(self) -> bool:
        if self.__condition is not None:
            return bool(self.__condition)
        else:
            return True


class ActionTransition(ConditionalTransition):
    Action = Callable[[], None]

    def __init__(self, next_state: 'State' = None) -> None:
        super().__init__()
        if isinstance(next_state, State) or next_state is None:
            self.__next_state = next_state
        else:
            raise TypeError("Le prochain état doit être de type State.")
        self.__transiting_actions: List['ActionTransition.Action'] = list()

    def _do_transiting_action(self) -> None:
        for transiting_action in self.__transiting_actions:
            transiting_action()

    def add_transiting_action(self, action: 'ActionTransition.Action') -> None:
        if callable(action):
            self.__transiting_actions.append(action)
        else:
            raise TypeError("L'action de transition doit être un Callable.")


class MonitoredTransition(ActionTransition):
    def __init__(self, next_state: 'State' = None) -> None:
        if isinstance(next_state, State) or next_state is None:
            super().__init__(next_state)
        else:
            raise TypeError("Le prochain état doit être de type State.")
        self.__transit_count: int = 0
        self.__last_transit_time: float = 0
        self.custom_value: any

    @property
    def transit_count(self) -> int:
        return self.__transit_count

    @property
    def last_transit_time(self) -> float:
        return self.__last_transit_time

    def reset_transit_count(self) -> None:
        self.__transit_count = 0

    def reset_last_transit_time(self) -> None:
        self.__last_transit_time = 0.0

    def _exec_transiting_action(self) -> None:
        start_time = perf_counter()
        self._do_transiting_action()
        self.__transit_count += 1
        self.__last_transit_time = perf_counter() - start_time


#########################


""" LEVEL 02: CONDITIONS """


class Condition(ABC):
    def __init__(self, inverse: bool = False) -> None:
        if type(inverse) is bool:
            self.__inverse = inverse
        else:
            raise TypeError("La variable inverse doit être de type bool.")

    @abstractmethod
    def _compare(self) -> bool:
        pass

    def __bool__(self) -> bool:
        return self._compare() ^ self.__inverse


class AlwaysTrueCondition(Condition):
    def __init__(self, inverse: bool = False):
        super().__init__(inverse)

    def _compare(self):
        return True


class ValueCondition(Condition):
    def __init__(self, initial_value: any, expected_value: any, inverse: bool = False):
        super().__init__(inverse)
        self.expected_value = expected_value
        self.value = initial_value

    def _compare(self):
        return self.value == self.expected_value


class TimedCondition(Condition):
    def __init__(self, duration: float = 1, time_reference: float = None, inverse: bool = False):
        super().__init__(inverse)
        if type(duration) is float:
            self.__counter_duration = duration
        else:
            raise TypeError("La variable duration doit être de type float.")
        if type(time_reference) is float:
            self.__counter_reference = time_reference
        elif time_reference is None:
            self.__counter_reference = perf_counter()
        else:
            raise TypeError("La variable time_reference doit être de type float ou None.")

    def _compare(self):
        return perf_counter() - self.__counter_reference >= self.__counter_duration

    @property
    def duration(self) -> float:
        return self.__counter_duration

    @duration.setter
    def duration(self, value: float) -> None:
        if type(value) is float:
            self.__counter_duration = value
        else:
            raise TypeError("La variable duration doit être de type float")

    def reset(self):
        self.__counter_reference = perf_counter()


########

ConditionList: List[Condition] = []


class ManyConditions(Condition):
    def __init__(self, inverse: bool = False):
        super().__init__(inverse)
        self._conditions: List[Condition] = ConditionList

    def add_condition(self, condition: Condition):
        if isinstance(condition, Condition):
            self._conditions.append(condition)
        else:
            raise TypeError("La condition doit être de type Condition")

    def add_conditions(self, conditions: ConditionList):
        for condition in conditions:
            if isinstance(condition, Condition):
                self._conditions.append(condition)
            else:
                raise TypeError("Les conditions doivent être de type Condition")


class AllCondition(ManyConditions):
    def __init__(self, inverse: bool = False):
        super().__init__(inverse)
        self.__inverse: bool = inverse

    def _compare(self) -> bool:
        for condition in self._conditions:
            if not bool(condition):
                return False
        return True


class NoneCondition(ManyConditions):
    def __init__(self, inverse: bool = False):
        super().__init__(inverse)

    def _compare(self) -> bool:
        for condition in self._conditions:
            if bool(condition):
                return False
        return True



class AnyCondition(ManyConditions):
    def __init__(self, inverse: bool = False):
        super().__init__(inverse)

    def _compare(self) -> bool:
        for condition in self._conditions:
            if bool(condition):
                return True
        return False



''' MonitoredStateCondition '''


class MonitoredStateCondition(Condition, ABC):
    def __init__(self, monitored_state: MonitoredState, inverse: bool = False):
        super().__init__(inverse)
        if isinstance(monitored_state, MonitoredState):
            self._monitored_state: MonitoredState = monitored_state
        else:
            raise TypeError("La variable monitored_state doit être de type MonitoredState")

    """ ACCESSEURS """

    @property
    def monitored_state(self) -> MonitoredState:
        return self._monitored_state

    """ MUTATEURS """

    @monitored_state.setter
    def monitored_state(self, new_monitored_state):
        if isinstance(new_monitored_state, MonitoredState):
            self._monitored_state = new_monitored_state
        else:
            raise TypeError(
                "L'argument transmis: new_monitored_state doit être une instance de la classe MonitoredState.")


class StateEntryDurationCondition(MonitoredStateCondition):
    def __init__(self, duration: float, monitored_state: MonitoredState, inverse: bool = False):
        super().__init__(monitored_state, inverse)
        ''' le temps requis à passer dans un certain State avant de déclencher transition vers le prochain State '''
        if type(duration) is float:
            self.__duration: float = duration
        else:
            raise TypeError("La variable duration doit être de type float.")

    """ ACCESSEURS """

    @property
    def duration(self) -> float:
        return self.__duration


    """ MUTATEURS """

    @duration.setter
    def duration(self, new_duration: float):
        if type(new_duration) is float:
            self.__duration = new_duration
        else:
            raise TypeError("Le paramètre doit être typé 'float'.")

    """ MÉTHODES """

    ''' inverse géré implicitement par magic function __call__ de la classe Condition '''
    ''' condition véritable lorsque le State a dépassé le temps d'activité minimal et requis avant la transition '''

    def _compare(self) -> bool:
        timer = perf_counter() - self._monitored_state.last_entry_time # MonitorState
        return timer >= self.__duration


class StateEntryCountCondition(MonitoredStateCondition):
    def __init__(
            self
            , expected_count: int
            , monitored_state: MonitoredState
            , auto_reset: bool = True
            , inverse: bool = False
    ):
        super().__init__(monitored_state, inverse)
        if type(auto_reset) is bool:
            self.__auto_reset: bool = auto_reset
        else:
            raise TypeError("La variable auto_reset doit être de type bool")
        ''' référence sur le nombre de fois que l'état à requis une transition ( incrémentation ) '''
        self.__ref_count: int = 0
        ''' compte de requêtes nécessaires avant la transition '''
        if type(expected_count) is int:
            self.__expected_count: int = expected_count
        else:
            raise TypeError("La variable expected_count doit être de type int")

    """ ACCESSEURS """

    @property
    def expected_count(self) -> int:
        return self.__expected_count

    """ MUTATEURS """

    @expected_count.setter
    def expected_count(self, expected_count: int):
        if type(expected_count) is int:
            self.__expected_count = expected_count
        else:
            raise TypeError("Le paramètre doit être typé 'int'.")

    """ MÉTHODES """

    ''' inverse géré implicitement par magic function __call__ de la classe Condition '''
    ''' condition véritable lorsque le nombre de fois que le State sollicite la transition dépasse le nombre '''

    def _compare(self) -> bool:
        self.__ref_count = self.monitored_state.entry_count
        count_reached = self.__ref_count >= self.__expected_count
        if self.__auto_reset and count_reached:
            self.reset_count()
        return count_reached

    def reset_count(self):
        self.monitored_state.reset_entry_count()


class StateValueCondition(MonitoredStateCondition):
    def __init__(self, expected_value: any, monitored_state: MonitoredState, inverse: bool = False):
        super().__init__(monitored_state, inverse)
        ''' self.__expected_value strictement équivalente à MonitoredState's custom_value propriété '''
        if type(expected_value) is type(monitored_state.custom_value):
            self.__expected_value: any = expected_value
        else:
            raise TypeError("Le type de la variable expected_value doit être comparable \
                à custom_value du MonitoredState")

    """ ACCESSEURS """

    @property
    def expected_value(self) -> any:
        return self.__expected_value

    """ MUTATEURS """

    @expected_value.setter
    def expected_value(self, new_expected_value: any):
        if type(new_expected_value) is type(self.monitored_state.custom_value):
            self.__expected_value = new_expected_value
        else:
            raise TypeError("Le type de la variable expected_value doit être comparable \
                à custom_value du MonitoredState")

    """ MÉTHODES """

    ''' inverse géré implicitement par magic function __call__ de la classe Condition '''
    ''' condition véritable lorsque self.expected_value est strictement équivalent à MonitoredState.custom_value '''

    def _compare(self) -> bool:
        return self.__expected_value == self._monitored_state.custom_value


In [491]:

class TextStateGenerator():
    def __init__(self, text_in = '', text_out = ''):
        self.__text_in = text_in
        self.__text_out = text_out

    def __call__(self) -> MonitoredState:
        state = MonitoredState()
#         state.add_entering_action(lambda :print(self.__text_in))
        # state.add_in_state_action(lambda: print("in state"))
#         state.add_exiting_action(lambda :print(self.__text_out))
        return state

class Blinker(FiniteStateMachine):
    StateGenerator = Callable[[], None]
    
    def __init__(self, state_generator: StateGenerator):
        self.__off = state_generator()
        self.__on = state_generator()

        ''' on_duration (inscription de la Transition; initialement, la duration est 0.0 - définie dans turn_on_1()) '''
        self.__on_duration = state_generator()
        self.t_on_duration = ConditionalTransition(StateEntryDurationCondition(.0, self.__on_duration))
        self.t_on_duration.next_state = self.__off
        self.__on_duration.add_transition(self.t_on_duration)

        ''' off_duration (semblable à on_duration) '''
        self.__off_duration = state_generator()
        self.t_off_duration = ConditionalTransition(StateEntryDurationCondition(.0, self.__off_duration))
        self.t_off_duration.next_state = self.__on
        self.__off_duration.add_transition(self.t_off_duration)
        # initialiser une Transition
        # déterminer le prochain état après la Transition effectuée (le State self.__on est le prochain ici)
        # ne pas oublier de dé-commenter plus bas au moment d'ajouter le State dans la liste du Layout

        ''' blink_1 '''
        self.__blink_begin = MonitoredState()
        self.__blink_begin.custom_value = True
        
        self.__blink_off = state_generator()
        self.__blink_on = state_generator()

        self.t_blink_begin_0 = ConditionalTransition(StateValueCondition(True, self.__blink_begin))
        self.t_blink_begin_1 = ConditionalTransition(StateValueCondition(False, self.__blink_begin))
        self.t_blink_begin_0.next_state = self.__blink_on
        self.t_blink_begin_1.next_state = self.__blink_off
        self.__blink_begin.add_transition(self.t_blink_begin_0)
        self.__blink_begin.add_transition(self.t_blink_begin_1)

        self.t_blink_on = ConditionalTransition(StateEntryDurationCondition(.0, self.__blink_on))
        self.t_blink_on.next_state = self.__blink_off
        self.__blink_on.add_transition(self.t_blink_on)
        self.t_blink_off = ConditionalTransition(StateEntryDurationCondition(.0, self.__blink_off))
        self.t_blink_off.next_state = self.__blink_on
        self.__blink_off.add_transition(self.t_blink_off)


        ''' blink_2, blink_3, blink_4 '''
        self.__blink_stop_begin = MonitoredState()
        self.__blink_stop_begin.custom_value = True
        self.__blink_stop_end = MonitoredState()
        self.__blink_stop_end.custom_value = True
        
        self.__blink_stop_off = state_generator()
        self.__blink_stop_on = state_generator()

        self.t_blink_stop_begin_0 = ConditionalTransition(StateValueCondition(True, self.__blink_stop_begin))
        self.t_blink_stop_begin_1 = ConditionalTransition(StateValueCondition(False, self.__blink_stop_begin))
        self.t_blink_stop_begin_0.next_state = self.__blink_stop_on
        self.t_blink_stop_begin_1.next_state = self.__blink_stop_off
        self.__blink_stop_begin.add_transition(self.t_blink_stop_begin_0)
        self.__blink_stop_begin.add_transition(self.t_blink_stop_begin_1)
        
        self.t_blink_stop_off_0 = ConditionalTransition(StateEntryDurationCondition(.0, self.__blink_stop_off))
        self.t_blink_stop_off_0.next_state = self.__blink_stop_on
        self.__blink_stop_off.add_transition(self.t_blink_stop_off_0)
        self.t_blink_stop_off_end = ConditionalTransition(StateEntryDurationCondition(.0, self.__blink_stop_begin))
        self.t_blink_stop_off_end.next_state = self.__blink_stop_end
        self.__blink_stop_off.add_transition(self.t_blink_stop_off_end)

        self.t_blink_stop_on_0 = ConditionalTransition(StateEntryDurationCondition(.0, self.__blink_stop_on))
        self.t_blink_stop_on_0.next_state = self.__blink_stop_off
        self.__blink_stop_on.add_transition(self.t_blink_stop_on_0)
        self.t_blink_stop_on_end = ConditionalTransition(StateEntryDurationCondition(.0, self.__blink_stop_begin))
        self.t_blink_stop_on_end.next_state = self.__blink_stop_end
        self.__blink_stop_on.add_transition(self.t_blink_stop_on_end)
        
        self.t__blink_stop_end_0 = ConditionalTransition(StateValueCondition(False, self.__blink_stop_end))
        self.t__blink_stop_end_1 = ConditionalTransition(StateValueCondition(True, self.__blink_stop_end))
        self.t__blink_stop_end_0.next_state = self.__on
        self.t__blink_stop_end_1.next_state = self.__off
        self.__blink_stop_end.add_transition(self.t__blink_stop_end_0)
        self.__blink_stop_end.add_transition(self.t__blink_stop_end_1)
        
        # Layout
        super().__init__(
            FiniteStateMachine.Layout(
                [
                    self.__off
                    , self.__on
                    , self.__on_duration
                    , self.__off_duration
                    , self.__blink_begin
                    , self.__blink_off
                    , self.__blink_on

                    , self.__blink_stop_begin
                    , self.__blink_stop_off
                    , self.__blink_stop_on
                    , self.__blink_stop_end
                ]
                , self.__off
            )
        )

    @property
    def is_on(self) -> bool:
        if self.current_applicative_state is self.__on:
            return True
        if self.current_applicative_state is self.__on_duration:
            return True
        if self.current_applicative_state is self.__blink_on:
            return True
        if self.current_applicative_state is self.__blink_stop_on:
            return True
        return False
      
      
    @property
    def is_off(self) -> bool:
        if self.current_applicative_state is self.__off:
            return True
        if self.current_applicative_state is self.__off_duration:
            return True
        if self.current_applicative_state is self.__blink_off:
            return True
        if self.current_applicative_state is self.__blink_stop_off:
            return True
        return False
    

    def turn_on_0(self):
        self.transit_to(self.__on)

    def turn_off_0(self):
        self.transit_to(self.__off)

    def turn_on_1(self, duration: float) -> None:
        self.__on_duration.transitions[0].condition.duration = duration
        self.transit_to(self.__on_duration) 

    def turn_off_1(self, duration: float) -> None:
        self.__off_duration.transitions[0].condition.duration = duration
        self.transit_to(self.__off_duration)        

    def blink1(self, cycle_duration: float = 1.0, percent_on: float = 0.5, begin_on: bool = True) -> None:
        self.__blink_begin.custom_value = begin_on
        self.__blink_off.transitions[0].condition.duration = cycle_duration - (cycle_duration * percent_on)
        self.__blink_on.transitions[0].condition.duration = cycle_duration * percent_on
        self.transit_to(self.__blink_begin)

           
    def blink2(self, total_duration: float, cycle_duration: float = 1.0, percent_on: float = 0.5, begin_on: bool = True,
               end_off: bool = True) -> None:
        self.__blink_stop_begin.custom_value = begin_on
        self.__blink_stop_off.transitions[0].condition.duration = cycle_duration - (cycle_duration * percent_on)
        self.__blink_stop_on.transitions[0].condition.duration = cycle_duration * percent_on
        self.__blink_stop_off.transitions[1].condition.duration = total_duration 
        self.__blink_stop_on.transitions[1].condition.duration = total_duration
        self.__blink_stop_end.custom_value = end_off
        self.transit_to(self.__blink_stop_begin)
        

    def blink3(self, total_duration: float, n_cycles: int, percent_on: float = 0.5, begin_on: bool = True,
               end_off: bool = True) -> None:
        if n_cycles != 0:
            cycle_duration = total_duration / float(n_cycles)
        else:
            raise ArithmeticError("Cannot divide by 0")
        
        self.__blink_stop_begin.custom_value = begin_on
        self.__blink_stop_off.transitions[0].condition.duration = cycle_duration - (cycle_duration * percent_on)
        self.__blink_stop_on.transitions[0].condition.duration = cycle_duration * percent_on
        self.__blink_stop_off.transitions[1].condition.duration = total_duration 
        self.__blink_stop_on.transitions[1].condition.duration = total_duration
        self.__blink_stop_end.custom_value = end_off
        self.transit_to(self.__blink_stop_begin)    
        
    
    def blink4(self, n_cycles: int, cycle_duration: float = 1.0, percent_on: float() = 0.5, begin_on: bool = True,
               end_off: bool = True) -> None:
        total_duration = cycle_duration * n_cycles
        self.__blink_stop_begin.custom_value = begin_on
        self.__blink_stop_off.transitions[0].condition.duration = cycle_duration - (cycle_duration * percent_on)
        self.__blink_stop_on.transitions[0].condition.duration = cycle_duration * percent_on
        self.__blink_stop_off.transitions[1].condition.duration = total_duration
        self.__blink_stop_on.transitions[1].condition.duration = total_duration
        self.__blink_stop_end.custom_value = end_off
        self.transit_to(self.__blink_stop_begin)


class SideBlinker(FiniteStateMachine):
    class Side(Enum):
        LEFT = 0
        RIGHT = 1
        BOTH = 2
        LEFT_RECIPROCAL = 3
        RIGHT_RECIPROCAL = 4

    def __init__(self, state_generator: Blinker.StateGenerator):
        self.__left_blinker = Blinker(state_generator)
        self.__right_blinker = Blinker(state_generator)
        self.__off = state_generator()
        self.__on = state_generator()

        ''' on_duration (inscription de la Transition; initialement, la duration est 0.0 - définie dans turn_on_1()) '''
        self.__on_duration = state_generator()
        self.t_on_duration = ConditionalTransition(StateEntryDurationCondition(.0, self.__on_duration))
        self.t_on_duration.next_state = self.__off
        self.__on_duration.add_transition(self.t_on_duration)

        ''' off_duration (semblable à on_duration) '''
        self.__off_duration = state_generator()
        self.t_off_duration = ConditionalTransition(StateEntryDurationCondition(.0, self.__off_duration))
        self.t_off_duration.next_state = self.__on
        self.__off_duration.add_transition(self.t_off_duration)
        # initialiser une Transition
        # déterminer le prochain état après la Transition effectuée (le State self.__on est le prochain ici)
        # ne pas oublier de dé-commenter plus bas au moment d'ajouter le State dans la liste du Layout

        ''' blink_1 '''
        self.__blink_begin = MonitoredState()
        self.__blink_begin.custom_value = True
        self.__blink_off = state_generator()
        self.__blink_on = state_generator()

        self.t_blink_begin_0 = ConditionalTransition(StateValueCondition(True, self.__blink_begin))
        self.t_blink_begin_1 = ConditionalTransition(StateValueCondition(False, self.__blink_begin))
        self.t_blink_begin_0.next_state = self.__blink_on
        self.t_blink_begin_1.next_state = self.__blink_off
        self.__blink_begin.add_transition(self.t_blink_begin_0)
        self.__blink_begin.add_transition(self.t_blink_begin_1)

        self.t_blink_on = ConditionalTransition(StateEntryDurationCondition(.0, self.__blink_on))
        self.t_blink_on.next_state = self.__blink_off
        self.__blink_on.add_transition(self.t_blink_on)
        self.t_blink_off = ConditionalTransition(StateEntryDurationCondition(.0, self.__blink_off))
        self.t_blink_off.next_state = self.__blink_on
        self.__blink_off.add_transition(self.t_blink_off)

        ''' blink_2, blink_3, blink_4 '''
        self.__blink_stop_begin = MonitoredState()
        self.__blink_stop_begin.custom_value = True
        self.__blink_stop_end = MonitoredState()
        self.__blink_stop_end.custom_value = True
        
        self.__blink_stop_off = state_generator()
        self.__blink_stop_on = state_generator()

        self.t_blink_stop_begin_0 = ConditionalTransition(StateValueCondition(True, self.__blink_stop_begin))
        self.t_blink_stop_begin_1 = ConditionalTransition(StateValueCondition(False, self.__blink_stop_begin))
        self.t_blink_stop_begin_0.next_state = self.__blink_stop_on
        self.t_blink_stop_begin_1.next_state = self.__blink_stop_off
        self.__blink_stop_begin.add_transition(self.t_blink_stop_begin_0)
        self.__blink_stop_begin.add_transition(self.t_blink_stop_begin_1)
        
        self.t_blink_stop_off_0 = ConditionalTransition(StateEntryDurationCondition(.0, self.__blink_stop_off))
        self.t_blink_stop_off_0.next_state = self.__blink_stop_on
        self.__blink_stop_off.add_transition(self.t_blink_stop_off_0)
        self.t_blink_stop_off_end = ConditionalTransition(StateEntryDurationCondition(.0, self.__blink_stop_begin))
        self.t_blink_stop_off_end.next_state = self.__blink_stop_end
        self.__blink_stop_off.add_transition(self.t_blink_stop_off_end)

        self.t_blink_stop_on_0 = ConditionalTransition(StateEntryDurationCondition(.0, self.__blink_stop_on))
        self.t_blink_stop_on_0.next_state = self.__blink_stop_off
        self.__blink_stop_on.add_transition(self.t_blink_stop_on_0)
        self.t_blink_stop_on_end = ConditionalTransition(StateEntryDurationCondition(.0, self.__blink_stop_begin))
        self.t_blink_stop_on_end.next_state = self.__blink_stop_end
        self.__blink_stop_on.add_transition(self.t_blink_stop_on_end)
        
        self.t__blink_stop_end_0 = ConditionalTransition(StateValueCondition(False, self.__blink_stop_end))
        self.t__blink_stop_end_1 = ConditionalTransition(StateValueCondition(True, self.__blink_stop_end))
        self.t__blink_stop_end_0.next_state = self.__on
        self.t__blink_stop_end_1.next_state = self.__off
        self.__blink_stop_end.add_transition(self.t__blink_stop_end_0)
        self.__blink_stop_end.add_transition(self.t__blink_stop_end_1)
        
        super().__init__(
        FiniteStateMachine.Layout(
            [
                self.__off
                , self.__on
                , self.__on_duration
                , self.__off_duration
                , self.__blink_begin
                , self.__blink_off
                , self.__blink_on
                , self.__blink_stop_begin
                , self.__blink_stop_off
                , self.__blink_stop_on
                , self.__blink_stop_end
            ]
            , self.__off
        )
    )
        

    ''' MÉTHODES '''

    @property
    def is_off(self, side: Side):
        if side is SideBlinker.Side.LEFT:
            return self.__left_blinker.is_off
        elif side is SideBlinker.Side.RIGHT:
            return self.__right_blinker.is_off
        elif side is SideBlinker.Side.BOTH:
            return self.__left_blinker.is_off and self.__right_blinker.is_off
        elif side is SideBlinker.Side.LEFT_RECIPROCAL:
            return self.__left_blinker.is_off and self.__right_blinker.is_on
        elif side is SideBlinker.Side.RIGHT_RECIPROCAL:
            return self.__left_blinker.is_on and self.__right_blinker.is_off
        else:
            return
        
    def is_on(self, side: Side):
        if side is SideBlinker.Side.LEFT:
            return self.__left_blinker.is_on
        elif side is SideBlinker.Side.RIGHT:
            return self.__right_blinker.is_on
        elif side is SideBlinker.Side.BOTH:
            return self.__left_blinker.is_on and self.__right_blinker.is_on
        elif side is SideBlinker.Side.LEFT_RECIPROCAL:
            return self.__left_blinker.is_on and self.__right_blinker.is_off
        elif side is SideBlinker.Side.RIGHT_RECIPROCAL:
            return self.__left_blinker.is_off and self.__right_blinker.is_on
        else:
            return
        
    def turn_off_0(self, side: Side):
        if side is SideBlinker.Side.LEFT:
            self.__left_blinker.turn_off_0()
        elif side is SideBlinker.Side.RIGHT:
            self.__right_blinker.turn_off_0()
        elif side is SideBlinker.Side.BOTH:
            self.__left_blinker.turn_off_0()
            self.__right_blinker.turn_off_0()
        elif side is SideBlinker.Side.LEFT_RECIPROCAL:
            self.__left_blinker.turn_off_0()
            self.__right_blinker.turn_on_0()
        elif side is SideBlinker.Side.RIGHT_RECIPROCAL:
            self.__right_blinker.turn_off_0()
            self.__left_blinker.turn_on_0()
        else:
            return 

    def turn_on_0(self, side: Side):
        if side is SideBlinker.Side.LEFT:
            self.__left_blinker.turn_on_0()
        elif side is SideBlinker.Side.RIGHT:
            self.__right_blinker.turn_on_0()
        elif side is SideBlinker.Side.BOTH:
            self.__left_blinker.turn_on_0()
            self.__right_blinker.turn_on_0()
        elif side is SideBlinker.Side.LEFT_RECIPROCAL:
            self.__left_blinker.turn_on_0()
            self.__right_blinker.turn_off_0()
        elif side is SideBlinker.Side.RIGHT_RECIPROCAL:
            self.__right_blinker.turn_on_0()
            self.__left_blinker.turn_off_0()
        else:
            return
            
    def turn_off_1(self, side: Side, duration: float):
        if side is SideBlinker.Side.LEFT:
            self.__left_blinker.turn_off_1(duration)
        elif side is SideBlinker.Side.RIGHT:
            self.__right_blinker.turn_off_1(duration)
        elif side is SideBlinker.Side.BOTH:
            self.__left_blinker.turn_off_1(duration)
            self.__right_blinker.turn_off_1(duration)
        elif side is SideBlinker.Side.LEFT_RECIPROCAL:
            self.__left_blinker.turn_off_1(duration)
            self.__right_blinker.turn_on_1(duration)
        elif side is SideBlinker.Side.RIGHT_RECIPROCAL:
            self.__right_blinker.turn_off_1(duration)
            self.__left_blinker.turn_on_1(duration)
        else:
            return
            
    def turn_on_1(self, side: Side, duration: float):
        if side is SideBlinker.Side.LEFT:
            self.__left_blinker.turn_on_1(duration)
        elif side is SideBlinker.Side.RIGHT:
            self.__right_blinker.turn_on_1(duration)
        elif side is SideBlinker.Side.BOTH:
            self.__left_blinker.turn_on_1(duration)
            self.__right_blinker.turn_on_1(duration)
        elif side is SideBlinker.Side.LEFT_RECIPROCAL:
            self.__left_blinker.turn_on_1(duration)
            self.__right_blinker.turn_off_1(duration)
        elif side is SideBlinker.Side.RIGHT_RECIPROCAL:
            self.__right_blinker.turn_on_1(duration)
            self.__left_blinker.turn_off_1(duration)
        else:
            return
            
    
    """ fonctions blink_x """
    
    def blink1(
            self
            , side: Side
            , cycle_duration: float = 1.
            , percent_on: float = .5
            , begin_on: bool = True
    ):
        if side is SideBlinker.Side.LEFT:
            self.__left_blinker.blink1(cycle_duration, percent_on, begin_on)
        elif side is SideBlinker.Side.RIGHT:
            self.__right_blinker.blink1(cycle_duration, percent_on, begin_on)
        elif side is SideBlinker.Side.BOTH:
            self.__left_blinker.blink1(cycle_duration, percent_on, begin_on)
            self.__right_blinker.blink1(cycle_duration, percent_on, begin_on)
        elif side is SideBlinker.Side.LEFT_RECIPROCAL:
            self.__left_blinker.blink1(cycle_duration, percent_on, begin_on)
            self.__right_blinker.blink1(cycle_duration, percent_on, not begin_on)
        elif side is SideBlinker.Side.RIGHT_RECIPROCAL:
            self.__right_blinker.blink1(cycle_duration, percent_on, begin_on)
            self.__left_blinker.blink1(cycle_duration, percent_on, not begin_on)
            
    def blink2(
            self
            , side: Side
            , total_duration: float
            , cycle_duration: float = 1.
            , percent_on: float = .5
            , begin_on: bool = True
            , end_off: bool = True
    ):
        if side is SideBlinker.Side.LEFT:
            self.__left_blinker.blink2(total_duration, cycle_duration, percent_on, begin_on, end_off)
        elif side is SideBlinker.Side.RIGHT:
            self.__right_blinker.blink2(total_duration, cycle_duration, percent_on, begin_on, end_off)
        elif side is SideBlinker.Side.BOTH:
            self.__left_blinker.blink2(total_duration, cycle_duration, percent_on, begin_on, end_off)
            self.__right_blinker.blink2(total_duration, cycle_duration, percent_on, begin_on, end_off)
        elif side is SideBlinker.Side.LEFT_RECIPROCAL:
            self.__left_blinker.blink2(total_duration, cycle_duration, percent_on, begin_on, end_off)
            self.__right_blinker.blink2(total_duration, cycle_duration, percent_on, not begin_on, not end_off)
        elif side is SideBlinker.Side.RIGHT_RECIPROCAL:
            self.__right_blinker.blink2(total_duration, cycle_duration, percent_on, begin_on, end_off)
            self.__left_blinker.blink2(total_duration, cycle_duration, percent_on, not begin_on, not end_off)
            
    def blink3(
            self
            , side: Side
            , total_duration: float
            , n_cycles: int
            , percent_on: float = .5
            , begin_on: bool = True
            , end_off: bool = True
    ):
        if side is SideBlinker.Side.LEFT:
            self.__left_blinker.blink3(total_duration, n_cycles, percent_on, begin_on, end_off)
        elif side is SideBlinker.Side.RIGHT:
            self.__right_blinker.blink3(total_duration, n_cycles, percent_on, begin_on, end_off)
        elif side is SideBlinker.Side.BOTH:
            self.__left_blinker.blink3(total_duration, n_cycles, percent_on, begin_on, end_off)
            self.__right_blinker.blink3(total_duration, n_cycles, percent_on, begin_on, end_off)
        elif side is SideBlinker.Side.LEFT_RECIPROCAL:
            self.__left_blinker.blink3(total_duration, n_cycles, percent_on, begin_on, end_off)
            self.__right_blinker.blink3(total_duration, n_cycles, percent_on, not begin_on, not end_off)
        elif side is SideBlinker.Side.RIGHT_RECIPROCAL:
            self.__right_blinker.blink3(total_duration, n_cycles, percent_on, begin_on, end_off)
            self.__left_blinker.blink3(total_duration, n_cycles, percent_on, not begin_on, not end_off)

    def blink4(
            self
            , side: Side
            , n_cycles: int
            , cycle_duration: float = 1.
            , percent_on: float = .5
            , begin_on: bool = True
            , end_off: bool = True
    ):
        if side is SideBlinker.Side.LEFT:
            self.__left_blinker.blink4(n_cycles, cycle_duration, percent_on, begin_on, end_off)
        elif side is SideBlinker.Side.RIGHT:
            self.__right_blinker.blink4(n_cycles, cycle_duration, percent_on, begin_on, end_off)
        elif side is SideBlinker.Side.BOTH:
            self.__left_blinker.blink4(n_cycles, cycle_duration, percent_on, begin_on, end_off)
            self.__right_blinker.blink4(n_cycles, cycle_duration, percent_on, begin_on, end_off)
        elif side is SideBlinker.Side.LEFT_RECIPROCAL:
            self.__left_blinker.blink4(n_cycles, cycle_duration, percent_on, begin_on, end_off)
            self.__right_blinker.blink4(n_cycles, cycle_duration, percent_on, not begin_on, not end_off)
        elif side is SideBlinker.Side.RIGHT_RECIPROCAL:
            self.__right_blinker.blink4(n_cycles, cycle_duration, percent_on, begin_on, end_off)
            self.__left_blinker.blink4(n_cycles, cycle_duration, percent_on, not begin_on, not end_off)

    def track(self) -> bool:
        self.__left_blinker.track()
        self.__right_blinker.track()
        return super().track()

In [492]:
import easygopigo3 as gpg

class LedBlinkers(SideBlinker):
    def __init__(self, state_generator: Blinker.StateGenerator, robot):
        super().__init__(state_generator)
        self.robot=robot
        print(self.robot)
    
    def track(self):
        if self.is_on(SideBlinker.Side.LEFT):
            self.robot.led_on(1)
        else:
            self.robot.led_off(1)
        if self.is_on(SideBlinker.Side.RIGHT):
            self.robot.led_on(0)
        else:
            self.robot.led_off(0)
        return super().track()

class EyeBlinkers(SideBlinker):
    def __init__(self, state_generator: Blinker.StateGenerator, robot):
        super().__init__(state_generator)
        self.robot = robot
        color = (0,0,255) # Blue
        self.robot.set_right_eye_color(color)
        self.robot.set_left_eye_color(color)
        
    def track(self):
        if self.is_on(SideBlinker.Side.LEFT):
            self.robot.open_left_eye()
        else:
            self.robot.close_left_eye()
        if self.is_on(SideBlinker.Side.RIGHT):
            self.robot.open_right_eye()
        else:
            self.robot.close_right_eye()
        return super().track()
    
    def set_left_eye_color(self, color):
        self.robot.set_left_eye_color(color) 
        
    def set_right_eye_color(self, color):
        self.robot.set_right_eye_color(color)
    
        
class MotionMotor(SideBlinker):
    def __init__(self, state_generator: Blinker.StateGenerator, robot):
        super().__init__(state_generator)
        self.robot=robot
    
    def track(self):
        pass
    



class Robot():
    def __init__(self) -> None:
        self.robot = gpg.EasyGoPiGo3()
        self.ledBlinkers = LedBlinkers(TextStateGenerator("Entering", "Exiting"), self.robot)
        self.eyeBlinkers = EyeBlinkers(TextStateGenerator("Entering", "Exiting"), self.robot)       
        
        self.remote_control_port = 'AD1'
        self.remote_control = self.robot.init_remote(port=self.remote_control_port)

In [493]:
class RemoteControlTransition(MonitoredTransition):
    def __init__(self, next_state = None ):
        super().__init__(next_state)

class RemoteControlCondition(ValueCondition):
    def __init__(self, initial_value, expected_value, robot, inverse = False):
        super().__init__(initial_value, expected_value, inverse)
        self.robot = robot

    def _compare(self):
        longuest_string = 20 # longueur arbitraire, ne sert que pour l'affichage
        print(f'\r{self.robot.remote_control.get_remote_code()}', sep='', end=' ' * longuest_string)
        return self.expected_value == self.robot.remote_control.get_remote_code()
    
class ManualControlStateMachine(FiniteStateMachine):
    def __init__(self, robot):
        self.robot = robot
            
        ''' LES STATE'''
        stop_state = MonitoredState()
        forward_state = MonitoredState()
        backward_state = MonitoredState()    
        rotate_right_state = MonitoredState()
        rotate_left_state = MonitoredState()
        
        ''' STOP STATE'''
            
        stop_condition_0 = RemoteControlCondition("", "right", self.robot)
        stop_condition_1 = RemoteControlCondition("", "up", self.robot)
        stop_condition_2 = RemoteControlCondition("", "left", self.robot)
        stop_condition_3 = RemoteControlCondition("", "down",  self.robot)
        
        stop_transition_0 = RemoteControlTransition()
        stop_transition_1 = RemoteControlTransition() 
        stop_transition_2 = RemoteControlTransition()
        stop_transition_3 = RemoteControlTransition()    
        
        stop_transition_0.condition = stop_condition_0
        stop_transition_1.condition = stop_condition_1
        stop_transition_2.condition = stop_condition_2
        stop_transition_3.condition = stop_condition_3
        
        stop_transition_0.next_state = rotate_right_state
        stop_transition_1.next_state = forward_state
        stop_transition_2.next_state = rotate_left_state
        stop_transition_3.next_state = backward_state
        
        stop_state.add_transition(stop_transition_0)
        stop_state.add_transition(stop_transition_1)        
        stop_state.add_transition(stop_transition_2)
        stop_state.add_transition(stop_transition_3)
        stop_state.add_in_state_action(self.robot.robot.stop) 
        
        ''' FORWARD STATE''' 
        forward_condition_0 = RemoteControlCondition("up", "", self.robot)
        forward_transition_0 = RemoteControlTransition()
        forward_transition_0.condition = forward_condition_0
        forward_transition_0.next_state = stop_state
        forward_state.add_transition(forward_transition_0)
        forward_state.add_in_state_action(self.robot.robot.forward)
          

        ''' BACKWARD STATE'''
        backward_condition_0 = RemoteControlCondition("down", "", self.robot)
        backward_transition_0 = RemoteControlTransition()
        backward_transition_0.condition = backward_condition_0
        backward_transition_0.next_state = stop_state
        backward_state.add_transition(backward_transition_0)
        backward_state.add_in_state_action(self.robot.robot.backward)
        

        ''' ROTATE RIGHT STATE'''
        rotate_right_condition_0 = RemoteControlCondition("right", "", self.robot)
        rotate_right_transition_0 = RemoteControlTransition()
        rotate_right_transition_0.condition = rotate_right_condition_0
        rotate_right_transition_0.next_state = stop_state
        rotate_right_state.add_transition(rotate_right_transition_0)
        rotate_right_state.add_in_state_action(self.robot.robot.right)


        ''' ROTATE LEFT STATE'''
        rotate_left_condition_0 = RemoteControlCondition("left", "", self.robot)
        rotate_left_transition_0 = RemoteControlTransition()
        rotate_left_transition_0.condition = rotate_left_condition_0
        rotate_left_transition_0.next_state = stop_state
        rotate_left_state.add_transition(rotate_left_transition_0)
        rotate_left_state.add_in_state_action(self.robot.robot.left)

        ''' LAYOUT '''
        self.layout = FiniteStateMachine.Layout([stop_state, forward_state, backward_state, rotate_right_state, rotate_left_state], stop_state)
        super().__init__(self.layout)
      
    
class RobotState(MonitoredState):
    def __init__(self, robot):
        super().__init__()
        self.robot = robot
        
    
class ManualControlState(RobotState):
    def __init__(self, robot, manual_control_state_machine):
        super().__init__(robot)
        self.fsm = manual_control_state_machine
      
    
class C64Projet1(FiniteStateMachine):
    def __init__(self):
        self.robot = Robot()
        
        ''' TELECOMMANDE '''
        def read_input_home():
            home_state.custom_value = self.robot.remote_control.get_remote_code()
            
        ''' STATE '''
        home_state = MonitoredState()
        self.manual_control_fsm = ManualControlStateMachine(self.robot)
        manual_control_state = ManualControlState(self.robot, self.manual_control_fsm)
        
        ''' HOME '''
        home_state.custom_value = ""
        home_condition_0 = RemoteControlCondition("", "1", self.robot)
        home_transition_0 = RemoteControlTransition()
        home_transition_0.condition = home_condition_0
        home_transition_0.next_state = manual_control_state
        home_state.add_transition(home_transition_0)
        
        ''' MANUAL CONTROL '''


        manual_control_condition_0 = RemoteControlCondition("1", "ok", self.robot)
        manual_control_transition_0 = RemoteControlTransition()
        manual_control_transition_0.condition = manual_control_condition_0
        manual_control_transition_0.next_state = home_state
        manual_control_state.add_transition(manual_control_transition_0)
        manual_control_state.add_entering_action(self.manual_control_fsm.reset)
        manual_control_state.add_in_state_action(self.manual_control_fsm.track)
        manual_control_state.add_exiting_action(self.robot.robot.stop)
        
        ''' LAYOUT '''
        self.layout = FiniteStateMachine.Layout([home_state, manual_control_state], home_state)
        super().__init__(self.layout)
    
    def track(self) -> bool:
        self.robot.ledBlinkers.track()
        self.robot.eyeBlinkers.track()

        
#         longuest_string = 20 # longueur arbitraire, ne sert que pour l'affichage
#         print(f'\r{self.robot.ledBlinkers.is_on(SideBlinker.Side.LEFT)}', sep='', end=' ' * longuest_string)
#         self.eyesBlinker.track()
        return super().track()

In [494]:
# Test pour les blinkers
# projet1 = C64Projet1()
# # projet1.robot.ledBlinkers.blink4(SideBlinker.Side.LEFT_RECIPROCAL, cycle_duration=3, percent_on= 0.7, begin_on = True)
# # projet1.robot.eyeBlinkers.set_right_eye_color((0,255,0))
# # projet1.robot.eyeBlinkers.blink4(SideBlinker.Side.BOTH, n_cycles=3.0, cycle_duration=3.0, percent_on= 0.5, begin_on = True)

# projet1.start()

In [495]:
# test du ManualControlStateMachine
# robot = Robot()
# c = ManualControlStateMachine(robot)
# c.start()

In [ ]:
pro = C64Projet1()
pro.start()